# PrivacyScrub V4: Production Build & Deployment Pipeline
**Author:** Greg Burns
**Date:** November 20, 2025

## Executive Summary
This notebook serves as the master deployment controller for PrivacyScrub, a scalable media anonymization platform designed to redact Personally Identifiable Information (PII) from images and videos.

The system adheres to strict regulatory standards (GDPR, HIPAA, CCPA) by enforcing compliance profiles at the infrastructure level. It utilizes a distributed architecture where a central API orchestrates GPU-accelerated worker nodes to process high-definition video in parallel chunks.

### Architecture Overview
1.  **API Gateway (FastAPI):** Handles authentication, job lifecycle management, and request validation.
2.  **State Management (Firestore):** Persists job status and processing metrics.
3.  **Orchestration (Cloud Tasks):** Dispatches video segments to worker nodes for parallel processing.
4.  **Compute Engine (Cloud Run):** A containerized worker service that runs the multi-model detection stack (Face, Plate, Text, Logo).
5.  **Storage (GCS):** Holds raw inputs, intermediate chunks, and finalized outputs with strict Time-To-Live (TTL) policies.

### Deployment Automation
This notebook includes a fully automated CI/CD pipeline that:
* **Backend:** Builds a Docker container containing the AI models and FFmpeg binaries, pushes it to the Google Artifact Registry, and deploys it to Cloud Run.
* **Frontend:** Commits the reference UI code to GitHub, triggering an update to the hosted Streamlit interface.

## 1.0 Environment Initialization
This section configures the local environment, installs necessary Python dependencies for the build process, and authenticates with Google Cloud Platform.

In [ ]:
# Install production dependencies
# [FIX] Removed specific version pin for ultralytics to ensure compatibility with PyTorch 2.6+
!pip install -U -q fastapi[all] uvicorn python-multipart google-cloud-storage google-cloud-tasks google-cloud-firestore opencv-python-headless pillow ultralytics easyocr ffmpeg-python pydantic numpy

import os
import shutil
from google.colab import auth, userdata

print("Initializing Deployment Environment...")

# 1. Google Cloud Authentication
try:
    auth.authenticate_user()
    print("Authenticated successfully.")
except Exception as e:
    print(f"Authentication failed: {e}")

# 2. Load Configuration Secrets
# These must be set in the Colab Secrets manager for security.
try:
    PROJECT_ID = userdata.get('GCP_PROJECT_ID')
    REGION = userdata.get('GCP_REGION')
    BUCKET_NAME = userdata.get('GCS_BUCKET_NAME')
    SERVICE_NAME = userdata.get('SERVICE_NAME')
    GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')

    # Configure the gcloud CLI
    os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
    !gcloud config set project {PROJECT_ID}
    !gcloud config set run/region {REGION}

    # Enable necessary Google Cloud APIs
    !gcloud services enable cloudbuild.googleapis.com run.googleapis.com artifactregistry.googleapis.com cloudtasks.googleapis.com firestore.googleapis.com

    print(f"Environment configured for Project: {PROJECT_ID}")
except Exception as e:
    print("Missing secrets. Please ensure GCP_PROJECT_ID, GCP_REGION, GCS_BUCKET_NAME, SERVICE_NAME, and GITHUB_TOKEN are set.")
    raise e

# 3. Scaffolding
if os.path.exists("app"): shutil.rmtree("app")
os.makedirs("app/models", exist_ok=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.4 MB/s eta 0:00:00
Initializing Deployment Environment...
Authenticated successfully.
INFORMATION: Project 'privacyscrub-backend' has no 'environment' tag set. Use either 'Production', 'Development', 'Test', or 'Staging'. Add an 'environment' tag using `gcloud resource-manager tags bindings create`.
Updated property [core/project].
Updated property [run/region].
Operation "operations/acat.p2-138163390354-58d9aa7d-2487-40a5-ac00-c45a60043dce" finished successfully.
Environment configured for Project: privacyscrub-backend


## 2.0 Application Source Code
The following cells write the modular Python source code to the local filesystem. This structure constitutes the microservice that will be containerized and deployed to Cloud Run.

In [ ]:
%%writefile app/config.py
from enum import Enum
from typing import Optional, Tuple
from pydantic import BaseModel, Field, validator

class AnonymizeMode(str, Enum):
    BLUR = "blur"
    PIXELATE = "pixelate"
    BLACK_BOX = "black_box"

class ComplianceProfile(str, Enum):
    NONE = "NONE"
    GDPR = "GDPR"
    CCPA = "CCPA"
    HIPAA_SAFE_HARBOR = "HIPAA_SAFE_HARBOR"

class JobStatus(str, Enum):
    QUEUED = "QUEUED"
    CHUNKING = "CHUNKING"
    PROCESSING = "PROCESSING"
    STITCHING = "STITCHING"
    COMPLETED = "COMPLETED"
    FAILED = "FAILED"
    CANCELLED = "CANCELLED"

class PrivacyConfig(BaseModel):
    """
    Defines the target detection settings and redaction modes.
    Validates inputs to prevent invalid processing requests.
    """
    target_faces: bool = True
    target_plates: bool = True
    target_logos: bool = False
    target_text: bool = False
    mode: AnonymizeMode = AnonymizeMode.BLUR
    confidence_threshold: float = Field(0.5, ge=0.0, le=1.0)
    coordinates_only: bool = False
    strip_metadata: bool = True
    roi: Optional[Tuple[float, float, float, float]] = None

    @validator('roi', pre=True)
    def parse_roi(cls, v):
        # Parses comma-separated strings into a tuple of floats for ROI processing
        if isinstance(v, str) and v.strip():
            return tuple(map(float, v.split(',')))
        return v if isinstance(v, tuple) else None

def get_config_for_profile(profile: ComplianceProfile, user_config: PrivacyConfig) -> PrivacyConfig:
    """
    Enforces regulatory requirements by overriding user settings.
    For example, selecting 'HIPAA_SAFE_HARBOR' forces 'BLACK_BOX' mode and enables
    all detection targets to ensure maximum privacy.
    """
    cfg = user_config.model_copy()
    if profile == ComplianceProfile.GDPR:
        # GDPR mandates strict removal of personal identifiers
        cfg.confidence_threshold = max(cfg.confidence_threshold, 0.6)
        cfg.target_faces = True
        cfg.target_plates = True
        cfg.target_text = True
        cfg.strip_metadata = True
    elif profile == ComplianceProfile.CCPA:
        # CCPA focuses on household identifiers and personal data
        cfg.confidence_threshold = max(cfg.confidence_threshold, 0.55)
        cfg.target_faces = True
        cfg.target_plates = True
        cfg.strip_metadata = True
    elif profile == ComplianceProfile.HIPAA_SAFE_HARBOR:
        # HIPAA requires de-identification of 18 specific identifiers
        cfg.confidence_threshold = max(cfg.confidence_threshold, 0.7)
        cfg.mode = AnonymizeMode.BLACK_BOX
        cfg.strip_metadata = True
        cfg.target_faces = True
        cfg.target_plates = True
        cfg.target_text = True
        cfg.target_logos = True
    return cfg

Writing app/config.py


In [ ]:
%%writefile app/logger.py
import logging, json, sys
from datetime import datetime, timezone

class JsonFormatter(logging.Formatter):
    """
    Formats log records as JSON objects for ingestion by cloud observability tools.
    Strictly avoids logging PII or raw image data.
    """
    def format(self, record):
        log_obj = {
            "severity": record.levelname,
            "timestamp": datetime.now(timezone.utc).isoformat(),
            "message": record.getMessage(),
            "component": record.module,
            "job_id": getattr(record, "job_id", "system")
        }
        if hasattr(record, "metrics"): log_obj["metrics"] = record.metrics
        return json.dumps(log_obj)

logger = logging.getLogger("privacyscrub")
handler = logging.StreamHandler(sys.stdout)
handler.setFormatter(JsonFormatter())
logger.addHandler(handler)
logger.setLevel(logging.INFO)

def log(msg: str, severity: str = "INFO", job_id: str = None, **metrics):
    """Wrapper to enforce structured logging standards."""
    extra = {"metrics": metrics} if metrics else {}
    if job_id: extra["job_id"] = job_id
    if severity == "ERROR": logger.error(msg, extra=extra)
    elif severity == "DEBUG": logger.debug(msg, extra=extra)
    else: logger.info(msg, extra=extra)

Writing app/logger.py


In [ ]:
%%writefile app/inference.py
import re, torch
import numpy as np
from ultralytics import YOLO
import easyocr
from app.config import PrivacyConfig

# Shared model instances to optimize memory usage in containerized environments
SHARED_BACKBONE = None
SHARED_OCR = None

def get_backbone():
    global SHARED_BACKBONE
    if SHARED_BACKBONE is None: SHARED_BACKBONE = YOLO('yolov8x.pt')
    return SHARED_BACKBONE

def get_ocr():
    global SHARED_OCR
    if SHARED_OCR is None: SHARED_OCR = easyocr.Reader(['en'], gpu=torch.cuda.is_available())
    return SHARED_OCR

# --- Dedicated Detector Classes ---
# These classes encapsulate the logic for detecting specific types of sensitive information.

class FaceDetector:
    def detect(self, img, results, conf_thresh):
        dets = []
        # Extracts class 0 (Person) and applies geometric heuristics to isolate the face region
        for box in results[0].boxes:
            if int(box.cls[0]) == 0 and float(box.conf[0]) >= conf_thresh:
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                fh = int((y2 - y1) * 0.2)
                poly = [[x1, y1], [x2, y1], [x2, y1 + fh], [x1, y1 + fh]]
                dets.append({"type": "face", "poly": poly, "conf": float(box.conf[0])})
        return dets

class PlateDetector:
    def detect(self, img, results, conf_thresh):
        dets = []
        # Extracts Vehicle classes (2,3,5,7) and targets the license plate region
        for box in results[0].boxes:
            if int(box.cls[0]) in [2, 3, 5, 7] and float(box.conf[0]) >= conf_thresh:
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                ph = int((y2 - y1) * 0.25)
                poly = [[x1, y2 - ph], [x2, y2 - ph], [x2, y2], [x1, y2]]
                dets.append({"type": "plate", "poly": poly, "conf": float(box.conf[0])})
        return dets

class LogoDetector:
    def detect(self, img, results, conf_thresh):
        dets = []
        # Uses proxy classes (backpacks, suitcases) to identify branded objects
        for box in results[0].boxes:
            if int(box.cls[0]) in [24, 26, 28] and float(box.conf[0]) >= conf_thresh:
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                poly = [[x1, y1], [x2, y1], [x2, y2], [x1, y2]]
                dets.append({"type": "logo", "poly": poly, "conf": float(box.conf[0])})
        return dets

class TextDetector:
    PII = { "email": r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}",
            "phone": r"(\+\d{1,2}\s)?\(?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}",
            "ssn": r"\d{3}-\d{2}-\d{4}" }

    def detect(self, img, conf_thresh, target_all):
        dets = []
        # Runs Optical Character Recognition and checks for PII patterns or bulk text
        for (bbox, text, prob) in get_ocr().readtext(img):
            if prob >= conf_thresh:
                if target_all or any(re.search(p, text) for p in self.PII.values()):
                    dets.append({"type": "text", "poly": [[int(p[0]), int(p[1])] for p in bbox], "conf": prob, "text": text})
        return dets

def detect_frame_fusion(img, config: PrivacyConfig):
    """
    The primary entry point for detection. Orchestrates the various detector classes
    and aggregates their results into a unified list of bounding boxes.
    """
    h, w, _ = img.shape
    results = get_backbone().predict(img, conf=config.confidence_threshold, verbose=False)
    fused = []

    if config.target_faces: fused.extend(FaceDetector().detect(img, results, config.confidence_threshold))
    if config.target_plates: fused.extend(PlateDetector().detect(img, results, config.confidence_threshold))
    if config.target_logos: fused.extend(LogoDetector().detect(img, results, config.confidence_threshold))
    if config.target_text: fused.extend(TextDetector().detect(img, config.confidence_threshold, config.target_text))

    # Apply Region of Interest (ROI) filtering if configured
    if config.roi:
        rx1, ry1, rx2, ry2 = config.roi[0]*w, config.roi[1]*h, config.roi[2]*w, config.roi[3]*h
        fused = [d for d in fused if any(rx1<=p[0]<=rx2 and ry1<=p[1]<=ry2 for p in d['poly'])]
    return fused

Writing app/inference.py


In [ ]:
%%writefile app/media.py
import cv2, numpy as np, subprocess, ffmpeg, io, os
from PIL import Image
from app.config import PrivacyConfig, AnonymizeMode
from app.logger import log

def redact_frame(img, detections, config: PrivacyConfig):
    """
    Applies the selected anonymization effect (Blur, Pixelate, Black Box)
    to the regions identified by the detection engine.
    """
    out = np.array(img, copy=True) # Explicit copy to avoid read-only buffer errors
    h, w, _ = out.shape
    for det in detections:
        pts = np.array(det['poly'], np.int32)
        rect = cv2.boundingRect(pts)
        x, y, rw, rh = rect
        x, y = max(0, x), max(0, y)
        rw, rh = min(w-x, rw), min(h-y, rh)
        if rw <= 0 or rh <= 0: continue
        roi = out[y:y+rh, x:x+rw]

        if config.mode == AnonymizeMode.BLUR:
            k = max(3, rw//3) | 1
            out[y:y+rh, x:x+rw] = cv2.GaussianBlur(roi, (k,k), 30)
        elif config.mode == AnonymizeMode.BLACK_BOX:
            cv2.rectangle(out, (x, y), (x+rw, y+rh), (0,0,0), -1)
        elif config.mode == AnonymizeMode.PIXELATE:
            s = cv2.resize(roi, (max(1, rw//15), max(1, rh//15)), interpolation=cv2.INTER_LINEAR)
            out[y:y+rh, x:x+rw] = cv2.resize(s, (rw, rh), interpolation=cv2.INTER_NEAREST)
    return out

def process_nvenc(input_path, output_path, handler_func):
    """
    Wraps FFmpeg execution to use hardware acceleration (NVENC) where possible.
    It streams raw video frames through Python for redaction and pipes them back to FFmpeg for encoding.
    """
    cap = cv2.VideoCapture(input_path)
    w, h, fps = int(cap.get(3)), int(cap.get(4)), cap.get(5)
    try:
        # Attempt to initialize NVENC for GPU acceleration
        process = ffmpeg.input('pipe:', format='rawvideo', pix_fmt='bgr24', s=f'{w}x{h}') \
            .output(output_path, vcodec='h264_nvenc', pix_fmt='yuv420p', r=fps) \
            .overwrite_output().run_async(pipe_stdin=True)
    except:
        # Fallback to CPU encoding (libx264) if GPU is unavailable
        log("GPU/NVENC Unavailable. Falling back to libx264 (CPU).", severity="WARN")
        process = ffmpeg.input('pipe:', format='rawvideo', pix_fmt='bgr24', s=f'{w}x{h}') \
            .output(output_path, vcodec='libx264', pix_fmt='yuv420p', r=fps) \
            .overwrite_output().run_async(pipe_stdin=True)

    while cap.isOpened():
        ret, f = cap.read()
        if not ret: break
        # Apply the redaction logic
        processed_frame = handler_func(f)
        # Write frame to FFmpeg stdin pipe
        process.stdin.write(processed_frame.astype(np.uint8).tobytes())

    process.stdin.close()
    process.wait()
    cap.release()

def clean_metadata(img_bytes):
    """Strips EXIF and other metadata from images to ensure total anonymization."""
    img = Image.open(io.BytesIO(img_bytes))
    out = io.BytesIO()
    img.save(out, format=img.format) # Saving via Pillow drops EXIF by default
    return out.getvalue(), f"image/{img.format.lower()}"

Writing app/media.py


In [ ]:
%%writefile app/main.py
import os, json, uuid, time, shutil, glob, re, subprocess
from datetime import datetime, timedelta, timezone
from fastapi import FastAPI, File, UploadFile, Form, HTTPException, Security
from fastapi.security.api_key import APIKeyHeader
from fastapi.responses import Response, JSONResponse
from google.cloud import storage, tasks_v2, firestore
import google.auth

from app.config import PrivacyConfig, ComplianceProfile, JobStatus, get_config_for_profile
from app.inference import detect_frame_fusion
from app.media import redact_frame, process_nvenc, clean_metadata
from app.logger import log

app = FastAPI(title="PrivacyScrub API", version="4.7.1")

# Load Environment Variables
PROJECT_ID = os.environ.get("GCP_PROJECT_ID")
BUCKET = os.environ.get("GCS_BUCKET_NAME")
REGION = os.environ.get("GCP_REGION", "us-central1")
QUEUE = "privacyscrub-video-queue"
SERVICE_URL = os.environ.get("SERVICE_URL")
# Default to 'secret' to match the Streamlit frontend default
API_KEY = os.environ.get("API_KEY", "secret")

# Initialize Cloud Clients
db = firestore.Client(project=PROJECT_ID)
store = storage.Client(project=PROJECT_ID)
tasks = tasks_v2.CloudTasksClient()

# Authentication Middleware
api_header = APIKeyHeader(name="X-API-KEY", auto_error=True)
async def check_auth(key: str = Security(api_header)):
    if key != API_KEY:
        # Log the failure to Cloud Logging for debugging
        print(f"Auth Failed. Expected: {API_KEY[:2]}*** Received: {key[:2]}***")
        raise HTTPException(403, "Invalid API Key")

def dispatch(endpoint, payload):
    """Helper to dispatch tasks to Cloud Tasks queue."""
    if not SERVICE_URL: return
    parent = tasks.queue_path(PROJECT_ID, REGION, QUEUE)
    task = {
        "http_request": {
            "http_method": tasks_v2.HttpMethod.POST,
            "url": f"{SERVICE_URL}{endpoint}",
            "headers": {"Content-Type": "application/json"},
            "body": json.dumps(payload).encode()
        }
    }
    tasks.create_task(request={"parent": parent, "task": task})

def natural_sort_key(s):
    """Ensures file chunks are sorted numerically (1, 2, 10) not lexically (1, 10, 2)."""
    return [int(text) if text.isdigit() else text.lower() for text in re.split('([0-9]+)', s)]

# --- Public Endpoints ---

@app.post("/v1/anonymize-image", dependencies=[Security(check_auth)])
async def image_ep(
    file: UploadFile = File(...), profile: ComplianceProfile = Form(ComplianceProfile.NONE),
    roi: str = Form(None), coordinates_only: bool = Form(False)
):
    """Synchronous endpoint for anonymizing individual images."""
    try:
        content = await file.read()
        base_cfg = PrivacyConfig(roi=roi, coordinates_only=coordinates_only)
        cfg = get_config_for_profile(profile, base_cfg)

        import cv2, numpy as np
        nparr = np.frombuffer(content, np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

        dets = detect_frame_fusion(img, cfg)
        if coordinates_only:
            return JSONResponse({"detections": dets})

        redacted = redact_frame(img, dets, cfg)
        _, enc = cv2.imencode('.jpg', redacted)
        final, mime = clean_metadata(enc.tobytes())
        return Response(final, media_type=mime)
    except Exception as e:
        log(f"Image Error: {e}", severity="ERROR")
        raise HTTPException(500, str(e))

@app.post("/v1/anonymize-video", dependencies=[Security(check_auth)])
async def video_ep(file: UploadFile = File(...), profile: ComplianceProfile = Form(ComplianceProfile.NONE), coordinates_only: bool = Form(False)):
    """Initiates an asynchronous video processing job."""
    jid = f"job_{uuid.uuid4()}"
    b = store.bucket(BUCKET)
    b.blob(f"input/{jid}/original.mp4").upload_from_file(file.file, content_type=file.content_type)

    db.collection("jobs").document(jid).set({
        "status": JobStatus.QUEUED, "profile": profile,
        "coordinates_only": coordinates_only,
        "created_at": firestore.SERVER_TIMESTAMP,
        "chunks_total": 0, "chunks_completed": 0
    })
    dispatch("/internal/split-video", {"job_id": jid})
    return {"job_id": jid, "status": "QUEUED"}

@app.get("/v1/jobs/{job_id}", dependencies=[Security(check_auth)])
def status_ep(job_id: str):
    """Returns job status, progress, and output URL."""
    doc = db.collection("jobs").document(job_id).get()
    if not doc.exists: raise HTTPException(404)
    data = doc.to_dict()

    total = data.get("chunks_total", 0)
    comp = data.get("chunks_completed", 0)
    data["progress"] = round(comp / total, 2) if total > 0 else 0.0

    if data["status"] == JobStatus.COMPLETED:
        created = data.get("created_at")
        if created: data["ttl"] = (created + timedelta(hours=24)).isoformat()

    return data

@app.delete("/v1/jobs/{job_id}", dependencies=[Security(check_auth)])
def cancel_ep(job_id: str):
    """Cancels a running job."""
    db.collection("jobs").document(job_id).update({"status": JobStatus.CANCELLED})
    return {"status": "CANCELLED"}

# --- Internal Workers (Orchestrated by Cloud Tasks) ---

@app.post("/internal/split-video")
def split_worker(p: dict):
    """Splits the video into 5-minute chunks for distributed processing."""
    jid = p["job_id"]
    job_ref = db.collection("jobs").document(jid)
    if job_ref.get().to_dict().get("status") == JobStatus.CANCELLED: return

    try:
        local = f"/tmp/{jid}_orig.mp4"
        b = store.bucket(BUCKET)
        b.blob(f"input/{jid}/original.mp4").download_to_filename(local)

        out_pattern = f"/tmp/{jid}_%03d.mp4"
        subprocess.run([
            "ffmpeg", "-i", local, "-c", "copy", "-f", "segment",
            "-segment_time", "300", "-reset_timestamps", "1", out_pattern
        ], check=True)

        chunks = sorted(glob.glob(f"/tmp/{jid}_*.mp4"))
        job_ref.update({"status": JobStatus.CHUNKING, "chunks_total": len(chunks)})

        for i, path in enumerate(chunks):
            uri = f"input/{jid}/chunks/chunk_{i}.mp4"
            b.blob(uri).upload_from_filename(path)
            dispatch("/internal/process-chunk", {"job_id": jid, "chunk_id": i, "uri": uri})
            os.remove(path)
    except Exception as e:
        job_ref.update({"status": JobStatus.FAILED, "error_message": str(e)})

@app.post("/internal/process-chunk")
def process_worker(p: dict):
    """Processes a single video chunk using the detection engine."""
    jid, cid = p["job_id"], p["chunk_id"]
    job_ref = db.collection("jobs").document(jid)
    snap = job_ref.get().to_dict()
    if snap.get("status") == JobStatus.CANCELLED: return

    try:
        if snap["status"] != JobStatus.PROCESSING:
             job_ref.update({"status": JobStatus.PROCESSING})

        b = store.bucket(BUCKET)
        local_in = f"/tmp/{jid}_{cid}_in.mp4"
        local_out = f"/tmp/{jid}_{cid}_out"
        b.blob(p["uri"]).download_to_filename(local_in)

        cfg = get_config_for_profile(ComplianceProfile(snap.get("profile", "NONE")),
                                     PrivacyConfig(coordinates_only=snap.get("coordinates_only", False)))

        if cfg.coordinates_only:
             # Generate JSON Manifest only
             import cv2
             cap = cv2.VideoCapture(local_in)
             man = []
             idx = 0
             while cap.isOpened():
                 ret, f = cap.read()
                 if not ret: break
                 man.append({"frame": idx, "detections": detect_frame_fusion(f, cfg)})
                 idx += 1
             with open(local_out+".json", "w") as f: json.dump(man, f)
             b.blob(f"output/{jid}/chunks/chunk_{cid}.json").upload_from_filename(local_out+".json")
        else:
             # Render Redacted Video
             def h(f): return redact_frame(f, detect_frame_fusion(f, cfg), cfg)
             process_nvenc(local_in, local_out+".mp4", h)
             b.blob(f"output/{jid}/chunks/chunk_{cid}.mp4").upload_from_filename(local_out+".mp4")

        job_ref.update({"chunks_completed": firestore.Increment(1)})

        curr = job_ref.get().to_dict()
        if curr["chunks_completed"] >= curr["chunks_total"]:
            dispatch("/internal/stitch-video", {"job_id": jid})

        if os.path.exists(local_in): os.remove(local_in)
    except Exception as e:
        log(f"Chunk Fail: {e}", "ERROR", jid)
        job_ref.update({"status": JobStatus.FAILED, "error_message": str(e)})

@app.post("/internal/stitch-video")
def stitch_worker(p: dict):
    """Combines processed chunks into the final output and strips metadata."""
    jid = p["job_id"]
    job_ref = db.collection("jobs").document(jid)
    try:
        job_ref.update({"status": JobStatus.STITCHING})
        b = store.bucket(BUCKET)

        blobs = list(b.list_blobs(prefix=f"output/{jid}/chunks/"))
        blobs.sort(key=lambda x: natural_sort_key(x.name))

        if blobs[0].name.endswith(".json"):
            full = []
            for blob in blobs: full.extend(json.loads(blob.download_as_text()))
            dest = b.blob(f"output/{jid}/final.json")
            dest.upload_from_string(json.dumps(full))
        else:
            with open("inputs.txt", "w") as f:
                for blob in blobs:
                    local = f"/tmp/{os.path.basename(blob.name)}"
                    blob.download_to_filename(local)
                    f.write(f"file '{local}'\n")

            subprocess.run([
                "ffmpeg", "-y", "-f", "concat", "-safe", "0", "-i", "inputs.txt",
                "-map_metadata", "-1", # Strict Metadata Stripping
                "-c", "copy", f"/tmp/{jid}_final.mp4"
            ], check=True)
            dest = b.blob(f"output/{jid}/final.mp4")
            dest.upload_from_filename(f"/tmp/{jid}_final.mp4")

        url = dest.generate_signed_url(version="v4", expiration=timedelta(hours=1), method="GET")
        job_ref.update({"status": JobStatus.COMPLETED, "output_url": url})
    except Exception as e:
        job_ref.update({"status": JobStatus.FAILED, "error_message": str(e)})

Overwriting app/main.py


## 3.0 Backend Deployment (Google Cloud Run)
This section automates the deployment of the API. It creates a `Dockerfile`, builds the container image on Google Cloud Build, and deploys it to the Cloud Run serverless platform.

In [ ]:
%%writefile Dockerfile
FROM python:3.9-slim

# Install system dependencies for OpenCV and FFmpeg
RUN apt-get update && apt-get install -y libgl1 libglib2.0-0 ffmpeg && rm -rf /var/lib/apt/lists/*

WORKDIR /app

# Install Python dependencies
# [FIX] Removed version pin on ultralytics to resolve PyTorch 2.6+ weights_only=True error
RUN pip install --no-cache-dir fastapi uvicorn python-multipart google-cloud-storage google-cloud-tasks google-cloud-firestore opencv-python-headless pillow ultralytics easyocr ffmpeg-python

# Copy application code
COPY app /app/app

# Run the FastAPI server
CMD ["uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "8080"]

Overwriting Dockerfile


In [ ]:
print("Starting Automated Deployment Pipeline...")

# --- Step 1: Backend Build & Deploy ---
IMAGE_TAG = f"gcr.io/{PROJECT_ID}/{SERVICE_NAME}"

print(f"Building Container: {IMAGE_TAG}")
!gcloud builds submit --tag {IMAGE_TAG}

print("Deploying to Cloud Run...")
# Explicitly setting API_KEY to 'secret' to match frontend default
!gcloud run deploy {SERVICE_NAME} \
  --image {IMAGE_TAG} \
  --region {REGION} \
  --platform managed \
  --allow-unauthenticated \
  --set-env-vars=GCP_PROJECT_ID={PROJECT_ID},GCP_REGION={REGION},GCS_BUCKET_NAME={BUCKET_NAME},API_KEY=secret \
  --memory=4Gi \
  --cpu=2 \
  --timeout=900

# --- Step 2: Capture URL & Wire Frontend ---
import subprocess, shutil
try:
    service_url = subprocess.check_output(
        f"gcloud run services describe {SERVICE_NAME} --region {REGION} --format 'value(status.url)'",
        shell=True
    ).decode().strip()

    print(f"✅ Backend Active at: {service_url}")

    # Update backend env var for self-reference
    !gcloud run services update {SERVICE_NAME} --region {REGION} --set-env-vars=SERVICE_URL={service_url}

    # --- Step 3: Inject Configuration into Frontend ---
    print("Wiring Frontend to Backend...")

    with open("streamlit_app.py", "r") as f:
        app_code = f.read()

    # HARDCODING: Replace placeholder with actual values
    app_code = app_code.replace("REPLACE_WITH_SERVICE_URL", service_url)

    with open("streamlit_app.py", "w") as f:
        f.write(app_code)

    # --- Step 4: Push to GitHub ---
    print("Deploying to GitHub...")
    REPO_OWNER = "BURNSGREGM"
    REPO_NAME = "privacyscrub-frontend"
    REPO_URL = f"https://{GITHUB_TOKEN}@github.com/{REPO_OWNER}/{REPO_NAME}.git"

    if os.path.exists(REPO_NAME): shutil.rmtree(REPO_NAME)
    !git clone {REPO_URL}

    shutil.copy("streamlit_app.py", f"{REPO_NAME}/streamlit_app.py")

    %cd {REPO_NAME}
    !git config user.email "deploy-bot@privacyscrub.ai"
    !git config user.name "Deployment Bot"
    !git add streamlit_app.py
    !git commit -m "Auto-Wiring: Injected Production API URL"
    !git push origin main
    %cd ..

    print("✅ FULL DEPLOYMENT COMPLETE.")
    print("Streamlit Cloud will auto-update shortly. No manual configuration required.")

except Exception as e:
    print(f"Deployment Failed: {e}")

Starting Automated Deployment Pipeline...
Building Container: gcr.io/privacyscrub-backend/privacyscrub-api
Creating temporary archive of 79 file(s) totalling 56.0 MiB before compression.
Uploading tarball of [.] to [gs://privacyscrub-backend_cloudbuild/source/1763668301.587263-2cd33d80be104348a5988b086bf12678.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/privacyscrub-backend/locations/global/builds/8895432e-3726-4b89-b2fc-5c39b409f89f].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/8895432e-3726-4b89-b2fc-5c39b409f89f?project=138163390354 ].
Waiting for build to complete. Polling interval: 1 second(s).
 REMOTE BUILD OUTPUT
starting build "8895432e-3726-4b89-b2fc-5c39b409f89f"

FETCHSOURCE
Fetching storage object: gs://privacyscrub-backend_cloudbuild/source/1763668301.587263-2cd33d80be104348a5988b086bf12678.tgz#1763668311595239
Copying gs://privacyscrub-backend_cloudbuild/source/1763668301.587263-2cd33d80be104348a5988b086bf12678.tgz#1763668311

## 4.0 Frontend Deployment (Streamlit Cloud)
This section generates the reference User Interface code and pushes it to the GitHub repository `BURNSGREGM/privacyscrub-frontend`. Streamlit Cloud listens to this repository and will automatically redeploy the app upon receiving the commit.

In [ ]:
import os, subprocess, shutil

print("🔄 Starting Frontend Repair Pipeline...")

# 1. Validation: Ensure Secrets are Loaded
from google.colab import userdata
try:
    PROJECT_ID = userdata.get('GCP_PROJECT_ID')
    REGION = userdata.get('GCP_REGION')
    SERVICE_NAME = userdata.get('SERVICE_NAME')
    GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
except:
    print("❌ Error: Colab Secrets are missing. Please set GCP_PROJECT_ID, etc.")
    raise

# 2. Fetch the EXISTING Cloud Run URL
print("🔍 Locating Cloud Run Service...")
try:
    service_url = subprocess.check_output(
        f"gcloud run services describe {SERVICE_NAME} --project {PROJECT_ID} --region {REGION} --format 'value(status.url)'",
        shell=True
    ).decode().strip()

    if not service_url:
        raise ValueError("Service URL is empty. Is the backend deployed?")

    print(f"✅ Found Active Backend: {service_url}")

except Exception as e:
    print(f"❌ Could not find backend service: {e}")
    print("👉 Please run the 'Backend Deployment' cell first!")
    raise

# 3. Dynamically Write streamlit_app.py (No Placeholders!)
print("📝 Generating wired application file...")

app_code = f"""
import streamlit as st
import requests, time

# --- INFRASTRUCTURE CONFIGURATION (Auto-Generated) ---
API_URL = "{service_url}"
API_KEY = "secret"
# -----------------------------------------------------

st.set_page_config("PrivacyScrub Console", layout="wide")
st.title("PrivacyScrub Enterprise Console")

# Sidebar - Privacy Controls
st.sidebar.header("Privacy Controls")
profile = st.sidebar.selectbox("Compliance Profile", ["NONE", "GDPR", "CCPA", "HIPAA_SAFE_HARBOR"])
mode = st.sidebar.radio("Redaction Mode", ["blur", "pixelate", "black_box"])

# Sidebar - Granular Targets (Restored)
st.sidebar.subheader("Targets (Profile Override)")
t_faces = st.sidebar.checkbox("Faces", True)
t_plates = st.sidebar.checkbox("Plates", True)
t_logos = st.sidebar.checkbox("Logos", False)
t_text = st.sidebar.checkbox("Text (OCR)", False)

headers = {{"X-API-KEY": API_KEY}}
tab1, tab2 = st.tabs(["Single Image", "Video Job"])

with tab1:
    st.subheader("Image Anonymization")
    img = st.file_uploader("Upload Image", type=['jpg', 'png', 'jpeg'])
    if img and st.button("Process Image"):
        with st.spinner("Redacting PII..."):
            files = {{"file": img.getvalue()}}
            # Sending granular checkbox values to backend
            data = {{
                "profile": profile, "mode": mode,
                "target_faces": t_faces, "target_plates": t_plates,
                "target_logos": t_logos, "target_text": t_text
            }}
            try:
                r = requests.post(f"{{API_URL}}/v1/anonymize-image", headers=headers, files=files, data=data)
                if r.status_code == 200:
                    c1, c2 = st.columns(2)
                    c1.image(img, caption="Original")
                    c2.image(r.content, caption="Anonymized")
                else:
                    st.error(f"API Error: {{r.text}}")
            except Exception as e:
                st.error(f"Connection Error: {{e}}")

with tab2:
    st.subheader("Batch Video Processing")
    vid = st.file_uploader("Upload Video", type=['mp4'])
    if vid and st.button("Start Processing Job"):
        with st.spinner("Initializing Cloud Job..."):
            try:
                files = {{"file": vid.getvalue()}}
                data = {{"profile": profile}}
                r = requests.post(f"{{API_URL}}/v1/anonymize-video", headers=headers, files=files, data=data)
                if r.status_code == 200:
                    job_id = r.json()["job_id"]
                    st.success(f"Job Started: {{job_id}}")

                    status_ph = st.empty()
                    bar = st.progress(0)
                    while True:
                        time.sleep(3)
                        stat = requests.get(f"{{API_URL}}/v1/jobs/{{job_id}}", headers=headers).json()
                        s = stat['status']
                        p = stat.get('progress', 0.0)
                        status_ph.info(f"Status: {{s}} | Progress: {{int(p*100)}}%")
                        bar.progress(p)

                        if s == "COMPLETED":
                            st.success("Processing Complete!")
                            st.markdown(f"[Download Result]({{stat['output_url']}})")
                            break
                        if s in ["FAILED", "CANCELLED"]:
                            st.error(f"Job Failed: {{stat.get('error_message')}}")
                            break
                else:
                    st.error(f"API Error: {{r.text}}")
            except Exception as e:
                st.error(f"Connection Error: {{e}}")
"""

# Write to disk
with open("streamlit_app.py", "w") as f:
    f.write(app_code)

# 4. Push to GitHub
print("🚀 Deploying to GitHub...")
REPO_OWNER = "BURNSGREGM"
REPO_NAME = "privacyscrub-frontend"
REPO_URL = f"https://{GITHUB_TOKEN}@github.com/{REPO_OWNER}/{REPO_NAME}.git"

if os.path.exists(REPO_NAME): shutil.rmtree(REPO_NAME)
!git clone {REPO_URL}

shutil.copy("streamlit_app.py", f"{REPO_NAME}/streamlit_app.py")

%cd {REPO_NAME}
!git config user.email "deploy-bot@privacyscrub.ai"
!git config user.name "Deployment Bot"
!git add streamlit_app.py
!git commit -m "Fix: Dynamic URL Injection ({service_url})"
!git push origin main
%cd ..

print("✅ REPAIR COMPLETE.")
print(f"Streamlit App wired to: {service_url}")
print("Refresh your Streamlit Cloud tab in ~30 seconds.")

🔄 Starting Frontend Repair Pipeline...
🔍 Locating Cloud Run Service...
✅ Found Active Backend: https://privacyscrub-api-whbrskh54q-uc.a.run.app
📝 Generating wired application file...
🚀 Deploying to GitHub...
Cloning into 'privacyscrub-frontend'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 27 (delta 8), reused 24 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (27/27), 7.28 KiB | 7.28 MiB/s, done.
Resolving deltas: 100% (8/8), done.
/content/privacyscrub-frontend
[main cf77ede] Fix: Dynamic URL Injection (https://privacyscrub-api-whbrskh54q-uc.a.run.app)
 1 file changed, 84 insertions(+), 91 deletions(-)
 rewrite streamlit_app.py (73%)
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (2/2), done.
Writing objects: 100% (3/3), 657 bytes | 657.00 KiB/s, done.
Total 3 (delta 1), reused 2 (delta 

In [ ]:
%%writefile streamlit_app.py
import streamlit as st
import requests, time

# --- INFRASTRUCTURE CONFIGURATION (Injected by Deployment Script) ---
# The deployment script will replace this placeholder with the actual Cloud Run URL
API_URL = "REPLACE_WITH_SERVICE_URL"
API_KEY = "secret"
# --------------------------------------------------------------------

st.set_page_config("PrivacyScrub Console", layout="wide")
st.title("PrivacyScrub Enterprise Console")

# Sidebar - Privacy Controls
st.sidebar.header("Privacy Controls")
profile = st.sidebar.selectbox("Compliance Profile", ["NONE", "GDPR", "CCPA", "HIPAA_SAFE_HARBOR"])
mode = st.sidebar.radio("Redaction Mode", ["blur", "pixelate", "black_box"])

# Sidebar - Granular Targets (Restored)
st.sidebar.subheader("Targets (Profile Override)")
t_faces = st.sidebar.checkbox("Faces", True)
t_plates = st.sidebar.checkbox("Plates", True)
t_logos = st.sidebar.checkbox("Logos", False)
t_text = st.sidebar.checkbox("Text (OCR)", False)

headers = {"X-API-KEY": API_KEY}
tab1, tab2 = st.tabs(["Single Image", "Video Job"])

with tab1:
    st.subheader("Image Anonymization")
    img = st.file_uploader("Upload Image", type=['jpg', 'png', 'jpeg'])
    if img and st.button("Process Image"):
        if "REPLACE" in API_URL:
            st.error("⚠️ System not fully deployed. API URL is missing.")
        else:
            with st.spinner("Redacting PII..."):
                files = {"file": img.getvalue()}
                # Sending granular checkbox values to backend
                data = {
                    "profile": profile, "mode": mode,
                    "target_faces": t_faces, "target_plates": t_plates,
                    "target_logos": t_logos, "target_text": t_text
                }
                try:
                    r = requests.post(f"{API_URL}/v1/anonymize-image", headers=headers, files=files, data=data)
                    if r.status_code == 200:
                        c1, c2 = st.columns(2)
                        c1.image(img, caption="Original")
                        c2.image(r.content, caption="Anonymized")
                    else:
                        st.error(f"API Error: {r.text}")
                except Exception as e:
                    st.error(f"Connection Error: {e}")

with tab2:
    st.subheader("Batch Video Processing")
    vid = st.file_uploader("Upload Video", type=['mp4'])
    if vid and st.button("Start Processing Job"):
        if "REPLACE" in API_URL:
            st.error("⚠️ System not fully deployed. API URL is missing.")
        else:
            with st.spinner("Initializing Cloud Job..."):
                try:
                    files = {"file": vid.getvalue()}
                    # Sending profile (video endpoint uses profile defaults for targets)
                    data = {"profile": profile}
                    r = requests.post(f"{API_URL}/v1/anonymize-video", headers=headers, files=files, data=data)
                    if r.status_code == 200:
                        job_id = r.json()["job_id"]
                        st.success(f"Job Started: {job_id}")

                        status_ph = st.empty()
                        bar = st.progress(0)
                        while True:
                            time.sleep(3)
                            stat = requests.get(f"{API_URL}/v1/jobs/{job_id}", headers=headers).json()
                            s = stat['status']
                            p = stat.get('progress', 0.0)
                            status_ph.info(f"Status: {s} | Progress: {int(p*100)}%")
                            bar.progress(p)

                            if s == "COMPLETED":
                                st.success("Processing Complete!")
                                st.markdown(f"[Download Result]({stat['output_url']})")
                                break
                            if s in ["FAILED", "CANCELLED"]:
                                st.error(f"Job Failed: {stat.get('error_message')}")
                                break
                    else:
                        st.error(f"API Error: {r.text}")
                except Exception as e:
                    st.error(f"Connection Error: {e}")

Overwriting streamlit_app.py


In [ ]:
print("Deploying Frontend to GitHub...")

REPO_OWNER = "BURNSGREGM"
REPO_NAME = "privacyscrub-frontend"
REPO_URL = f"https://{GITHUB_TOKEN}@github.com/{REPO_OWNER}/{REPO_NAME}.git"

# 1. Clone Repository
if os.path.exists(REPO_NAME):
    shutil.rmtree(REPO_NAME)

try:
    !git clone {REPO_URL}

    # 2. Copy App File
    shutil.copy("streamlit_app.py", f"{REPO_NAME}/streamlit_app.py")

    # 3. Commit and Push
    %cd {REPO_NAME}
    !git config user.email "deploy-bot@privacyscrub.ai"
    !git config user.name "Deployment Bot"
    !git add streamlit_app.py
    !git commit -m "Auto-deploy from Production Notebook"
    !git push origin main
    %cd ..

    print("? Frontend successfully pushed to GitHub. Streamlit Cloud should auto-update shortly.")
except Exception as e:
    print(f"Frontend deployment failed: {e}")

Deploying Frontend to GitHub...
Cloning into 'privacyscrub-frontend'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 26 (delta 7), reused 24 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (26/26), 7.20 KiB | 7.21 MiB/s, done.
Resolving deltas: 100% (7/7), done.
/content/privacyscrub-frontend
[main d9d6642] Auto-deploy from Production Notebook
 1 file changed, 91 insertions(+), 84 deletions(-)
 rewrite streamlit_app.py (68%)
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 832 bytes | 832.00 KiB/s, done.
Total 3 (delta 1), reused 1 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
remote: This repository moved. Please use the new location:
remote:   https://github.com/burnsgregm/privacyscrub-frontend.git
To h